
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dhairya0907/Creepypasta-Text-Generator/blob/main/Scripts/Charcter_Level_Text_Generation.ipynb)

In [ ]:
%%capture
!pip install wandb -qqq

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow.compat.v2 as tf
from tensorflow.python.framework.ops import disable_eager_execution

tf.enable_v2_behavior()
disable_eager_execution()

In [ ]:
import os
import io
import time
import wandb
import random
import datetime
import numpy as np
import pandas as pd


from wandb import AlertLevel
from keras.layers import RNN
from tensorflow import keras
from keras.layers import LSTM
from google.colab import drive
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Dropout
from tensorflow.keras import layers
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.callbacks import ModelCheckpoint

In [ ]:
drive.mount('/content/drive')

In [ ]:
wandb.login()

In [ ]:
Epochs = 1
Batch = 128
time_folder = "First"
story_length = 100
vocab_Max = 34
vocab_Orignal = 0
main_folder_name = "" # Folder name which have all the files.
file_name  = "" # File name on which you want to train model.

In [ ]:
folder_file = file_name.split("/")

In [ ]:
text = (open("/content/drive/My Drive/path_to_file").read())
text = text.lower()

In [ ]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())


text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in words or not w.isalpha())

In [ ]:
text = ''.join([i for i in text if not i.isdigit()])

In [ ]:
encoded_string = text.encode("ascii", "ignore")
text = encoded_string.decode()

In [ ]:
bad_chars = ['~', '|', '\x11', '#', '$', '%','(', ')', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>','@', '\\', '^', '_', '`',]
 
# using replace() to
# remove bad_chars
for i in bad_chars :
    text = text.replace(i, '')

In [ ]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

vocab_size = len(characters)
print('Number of unique characters: ', vocab_size)
print(characters)

In [ ]:
vocab_Orignal = vocab_size

In [ ]:
# Optional

experiment_name = ""

wandb.init(project="", 
           name=folder_file[2],
           sync_tensorboard=True,
           group=experiment_name,
           notes="",
           tags=["", "", "","",""])


artifact = wandb.Artifact('', type='dataset')
artifact.add_file("/content/drive/My Drive/path_to_file")
wandb.log_artifact(artifact)

wandb.config.update({"epochs": Epochs, 
                     "batch_size": Batch,
                     "architecture": "RNN",
                     "dataset": file_name,
                     "activation": "softmax",
                     "optimizer": "Adam",
                     "loss": "categorical_crossentropy",
                     "metric": "accuracy",
                     "vocab_Max": vocab_Max,
                     "vocab_Orignal" : vocab_Orignal,
                     })


WANDB_DISABLE_CODE= "true"

In [ ]:
last = [' ', '!', '"', '&', "'", '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}'] # Vocab in last file
current =[' ', '!', '"', '&', "'", '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}'] # Vocab in current file

# Both should be same

In [ ]:
print(len(last))
print(len(current))

In [ ]:
list(set(current) - set(last))

In [ ]:
bad_chars = ""
# using replace() to
# remove bad_chars
for i in bad_chars :
    text = text.replace(i, '')

In [ ]:
list(set(last) - set(current))

In [ ]:
text = text + '}' + '{'

In [ ]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

vocab_size = len(characters)
print('Number of unique characters: ', vocab_size)
print(characters)

In [ ]:
import re
res = re.sub(' +', ' ', text)

In [ ]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

vocab_size = len(characters)
print('Number of unique characters: ', vocab_size)
print(characters)

In [ ]:
X = []   # extracted sequences
Y = []   # the target: follow up character for each sequence in X
length = len(text)
seq_length = 100

for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
    
print('Number of extracted sequences:', len(X))

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [ ]:
X_modified.shape, Y_modified.shape

In [ ]:
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

In [ ]:
model.summary()

In [ ]:
filename = "/content/drive/My Drive/path_to_last_best_model"
model.load_weights(filename) # After first run, for incremental training
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


filepath = "/content/drive/My Drive/Machine Learning Research/path_to_current_model/"+time_folder+"/"+os.path.splitext(folder_file[2])[0]+"-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [checkpoint, WandbCallback(), tensorboard_callback]

#model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# Optional

wandb.config.update({"layers" : len(model.layers),})

In [ ]:
start_time = time.time()

model.fit(X_modified, Y_modified, epochs=Epochs, batch_size=Batch, callbacks = callbacks_list)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
time_in_min = (time.time() - start_time)/60
print(time_in_min)

In [ ]:
time_in_min = round(time_in_min, 2)
print(time_in_min)

In [ ]:
time_in_hr = round(time_in_min, 2)/60
print(time_in_hr)

In [ ]:
time_in_hr = round(time_in_hr, 2)
print(time_in_hr)

In [ ]:
# Optional

wandb.config.update({"fitting_time" : str(time_in_hr)+"h"})

In [ ]:
#!kill 3639
%tensorboard --logdir logs/fit

In [ ]:
# load weights form best model just tranined

filename = ""
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.save("/content/drive/My Drive/path_to_final_model_for_this_run"+str(time_folder)+"/"+str(os.path.splitext(folder_file[2])[0])+"_final_model.hdf5")

In [ ]:
model.save("/content/drive/My Drive/path_to_final_model_for_this_run_for_algorithmia"+str(time_folder)+"/"+str(os.path.splitext(folder_file[2])[0])+"_algorithmia_model.h5")

In [ ]:
start = np.random.randint(0, len(X)-1) # or generate random start

string_mapped = list(X[start])

full_string = [n_to_char[value] for value in string_mapped]


print("Seed:")
print("\"", ''.join(full_string), "\"")


In [ ]:
filename = "/content/drive/My Drive/path_to_store_genrated_story"
if not os.path.exists(os.path.dirname(filename)):
  try:
    os.makedirs(os.path.dirname(filename))
  except OSError as exc:
    if exc.errno != errno.EEXIST:
      raise

f = open(filename, "a")
f.write(str(filename)+" : ")
f.write("\n")
f.write("\n")
f.write(str("First")+" : ")
f.write("\n")
f.write("\n")
f.write("Number of words : \n" + str(story_length))
f.write("\n")
f.write("\n")
s = "\"", ''.join(full_string), "\""
f.write("Starting String : \n "+str(s))
f.write("\n")
f.write("\n")

In [ ]:
def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:

# generating characters
for i in range(story_length):

  x = np.reshape(string_mapped, (1, len(string_mapped), 1))
  x = (x / float(len(characters))) + (np.random.rand(1, len(string_mapped), 1) * 0.01)
  prediction = model.predict(x, verbose=0)
  pred_index = sample(np.ndarray.flatten(prediction))
  seq = [n_to_char[value] for value in string_mapped]
  full_string.append(n_to_char[pred_index])
  string_mapped.append(pred_index)  # add the predicted character to the end
  string_mapped = string_mapped[1:len(string_mapped)] # shift the string one character forward by removing pos. 0

In [ ]:
txt=""
for char in full_string:
    txt = txt+char

In [ ]:
f.write("Story : \n")
f.write(txt)
f.write("\n")
f.write("\n")
f.write("\n")
f.write("\n")
f.close()

In [ ]:
print(start)
print(txt)

In [ ]:
# Optional

trained_model_artifact = wandb.Artifact(
            str(os.path.splitext("path_to_model", type="model",description="",)))


trained_model_artifact.add_file("/content/drive/My Drive/path_to_algorithmia_model"+str(os.path.splitext(folder_file[2])[0])+"_algorithmia_model.h5")
wandb.log_artifact(trained_model_artifact)

In [ ]:
# Optional

wandb.alert(
      title='Run  '+main_folder_name+'  Finished',
      text=f'The Run of file {file_name} is completed in {(time.time() - start_time)} seconds.',
      level=AlertLevel.INFO,
      wait_duration= datetime.timedelta(minutes=0)
    )

In [ ]:
# Optional

wandb.finish()